In [ ]:
import torch
import torch.nn as nn
#helps move data forwards in the function
import torch.nn.functional as F
import tensorflow as tf

In [ ]:
#model class inherits nn.Module
class Model(nn.Module):
  #input layer (antennastate, prxetc, phaserads)
  #hidden layer 1 (some neurons)
  #hidden layer 2 (some neurons)
  #output (x and y)
  def __init__(self, output_features = 1, input_features=3, l1 = 9, l2 = 10):
    #instantiate nn.module
    super().__init__()
    #out_x = 0
    #out_y = 0
    out_R = 0
    #fc = fully connected
    self.fc1 = nn.Linear(input_features, l1)
    self.fc2 = nn.Linear(l1, l2)
    self.out1 = nn.Linear(l2, output_features)
    #self.out2 = nn.Linear(l2, output_features)

    #training data labels - mark categories with likelihood percents
    #argmax(label) = category with most likelihood
    #for no categories feed in x and y coordinates for each sample


  def forward(self, x):
    #relu = rectified linear unit
    #if output < 0, correct to zero
    #x = F.relu(self.fc1(x))
    x = self.fc1(x)
    x = self.fc2(x)
    x = self.out1(x)
    #x2 = self.out2(x)
    #return x1, x2
    return x

In [ ]:
#manual seed for randomization
torch.manual_seed(4)
#create an instance of model
model = Model(input_features=4)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#githuburl = 'https://github.com/nikojlieto/firstnn/raw/main/Autofilled%20Placement.csv'
#githuburl2 = 'https://github.com/nikojlieto/antennann/raw/main/localization%20XLP%20no%20freeform%20or%20timestamp.csv'
#new data, new url
githuburl = "https://github.com/nikojlieto/antennann/raw/main/data/5tags-circle-antennasweep.csv"
#githuburl.encode('utf-8').strip()
my_df = pd.read_csv(githuburl)
#test_df = pd.read_csv(githuburl2)


In [ ]:
list(my_df.columns)

['id',
 'absolute_timestamp',
 'relative_timestamp',
 'interrogator_timestamp',
 'freeform',
 'rssi',
 'epc96',
 'doppler',
 'phase',
 'antenna',
 'rospecid',
 'channelindex',
 'tagseencount',
 'accessspecid',
 'inventoryparameterspecid',
 'lastseentimestamp',
 'antennastate']

In [ ]:
#split for training - X = features, y = outcome
#X = my_df[['antennastate', 'prxetc', 'phaserads']]
X = my_df[['phase', 'rssi', 'channelindex', 'antennastate']]
#y = my_df[['x', 'y']]
y = my_df[[]]

In [ ]:
#conversion to numpy arrays
X = X.values
y = y.values


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state =4)

In [ ]:
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
print(X_train.shape)
print(X_test.shape)
print(X_test)

torch.Size([82505, 4])
torch.Size([35360, 4])
tensor([[ 2.5800e+03, -3.4000e+01,  1.4000e+01,  3.0000e+00],
        [ 5.5200e+02, -3.4000e+01,  4.1000e+01,  0.0000e+00],
        [ 2.1720e+03, -5.4000e+01,  1.3000e+01,  2.0000e+00],
        ...,
        [ 1.9240e+03, -3.2000e+01,  3.0000e+01,  0.0000e+00],
        [ 1.5200e+03, -5.6000e+01,  4.9000e+01,  1.0000e+00],
        [ 2.1400e+03, -3.4000e+01,  3.6000e+01,  3.0000e+00]])


In [ ]:
y_train = torch.FloatTensor(y_train)
y_test = torch.FloatTensor(y_test)
y_train.shape

torch.Size([82505, 0])

In [ ]:
#set criterion of model to measure the error
criterion = nn.L1Loss()
#choose optimizer, lr = learning rate
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
#train model time oogie
#epoch - one run through of data
epochs = 100
#keep track of losses too
losses = []
#this should be a tensor instead of a tuple
#why is a single value being returned?

for i in range(100):
  #go forwards please
  #get predicted results here
  y_pred = model(X_train)
  #reformat from tuple
  y_pred = torch.stack([y_pred[0], y_pred[1]], axis=1)
  y_pred = torch.squeeze(y_pred)
  #measure the loss
  loss = criterion(y_pred, y_train)
  #to keep track
  losses.append(loss.detach().numpy())
  #print every 10 epoch
  if i%10 == 0:
    print(f'Epoch: {i} and loss: {loss}')
  #back propagation
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:101: UserWarning: Using a target size (torch.Size([82505, 0])) that is different to the input size (torch.Size([2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (2) must match the size of tensor b (0) at non-singleton dimension 1

In [ ]:
#saving model
torch.save(model.state_dict(), 'trainedfirstnn.pt')
#it can be loaded too

In [ ]:
#test data time
with torch.no_grad():
  #just some print statements to check what's going on
  #put in features get predictions
  y_eval = model.forward(X_test)
  #find the error
  #reshape results first
  y_eval = torch.stack([y_eval[0], y_eval[1]], axis=1)
  y_eval = torch.squeeze(y_eval)
  print(y_eval.shape)
  for i in range(1750):
    print (print("eval", y_eval[i], "\n test", y_test[i]))
  loss = criterion(y_eval, y_test)
  print(loss)

Streaming output truncated to the last 5000 lines.
None
eval tensor([-0.0242,  0.9921]) 
 test tensor([0., 1.])
None
eval tensor([0.0214, 0.9458]) 
 test tensor([0., 1.])
None
eval tensor([-0.0262,  0.9755]) 
 test tensor([0., 1.])
None
eval tensor([0.8040, 0.1097]) 
 test tensor([0., 0.])
None
eval tensor([0.9147, 0.0315]) 
 test tensor([1., 0.])
None
eval tensor([0.0286, 0.9441]) 
 test tensor([0., 1.])
None
eval tensor([-0.0288,  0.9727]) 
 test tensor([0., 1.])
None
eval tensor([0.0201, 0.9385]) 
 test tensor([0., 1.])
None
eval tensor([ 1.1414, -0.2267]) 
 test tensor([1., 0.])
None
eval tensor([0.9138, 0.0229]) 
 test tensor([1., 1.])
None
eval tensor([0.0337, 0.9142]) 
 test tensor([0., 1.])
None
eval tensor([-0.0286,  0.9954]) 
 test tensor([0., 1.])
None
eval tensor([-0.0220,  0.9722]) 
 test tensor([0., 1.])
None
eval tensor([0.0259, 0.9539]) 
 test tensor([0., 1.])
None
eval tensor([0.9121, 0.0173]) 
 test tensor([1., 1.])
None
eval tensor([-0.0265,  0.9762]) 
 test tensor([

In [ ]:
#new data eventually
test_columns =test_df[["antennastate", "prx linear ()", "phase (radians)"]]
test_row = test_columns.iloc[27]
tensor_row = torch.tensor(test_row.values)
tensor_row = tensor_row.to(torch.float32)
print(tensor_row.dtype)
for i in range(20):
  test_row = test_columns.iloc[i]
  tensor_row = torch.tensor(test_row.values)
  tensor_row = tensor_row.to(torch.float32)
  with torch.no_grad():
    print("row ", i)
    print(model(tensor_row))

torch.float32
row  0
(tensor([-0.8987]), tensor([1.9051]))
row  1
(tensor([-0.8959]), tensor([1.9168]))
row  2
(tensor([-0.8970]), tensor([1.8986]))
row  3
(tensor([-0.8877]), tensor([1.8932]))
row  4
(tensor([-0.8848]), tensor([1.8902]))
row  5
(tensor([-0.8899]), tensor([1.8952]))
row  6
(tensor([-0.8876]), tensor([1.8927]))
row  7
(tensor([-0.8876]), tensor([1.8928]))
row  8
(tensor([-0.8877]), tensor([1.8931]))
row  9
(tensor([-0.8875]), tensor([1.8924]))
row  10
(tensor([-0.8876]), tensor([1.8929]))
row  11
(tensor([-0.8981]), tensor([1.9032]))
row  12
(tensor([-0.8985]), tensor([1.9045]))
row  13
(tensor([-0.8957]), tensor([1.9160]))
row  14
(tensor([-0.9056]), tensor([1.9297]))
row  15
(tensor([-0.8960]), tensor([1.8955]))
row  16
(tensor([-0.8957]), tensor([1.9160]))
row  17
(tensor([-0.9057]), tensor([1.9302]))
row  18
(tensor([-0.8957]), tensor([1.9160]))
row  19
(tensor([-0.9058]), tensor([1.9306]))
(tensor([-0.9058]), tensor([1.9306]))
